# Menganalisis risiko gagal bayar peminjam
                                       
                                       

# 1. Pendahuluan

Kali ini projek yang akan dilakukan  adalah menyiapkan laporan untuk divisi kredit suatu bank. Dalam hal ini akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan ini nantinya akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

Dalam proyek berikut ini saya juga akan mencari tahu adakah pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam melunasi pinjaman di Bank.

# 2. Daftar Isi 

* [1. Pendahuluan]
* [2. Daftar Isi]
* [3. Tahapan]
    * [3.1 Pra-pemrosesan]
      * [3.1.1 Memuat Data]
      * [3.1.2 Mengeksplorasi Data Awal]
      * [3.1.3 Kesimpulan dan langkah-langkah]
    * [3.2 Menemukan pola nilai yang hilang]
    * [3.3 Transformasi data]
    * [3.4 Memperbaiki duplikasi data]
    * [3.5 Bekerja dengan nilai data]
       * [3.5.1 Memperbaiki nilai yang hilang di `total_income`]
       * [3.5.2 Memperbaiki nilai di `days_employed`
    * [3.6 Pengkategorian data]
    * [3.7 Memmeriksa hipotesis]
    * [3.8 Kesimpulan umum]

# 3. Tahapan

   ## 3.1 Pra-pemrosesan
   
   **Muat Data**

In [10]:
# Muat semua library
import pandas as pd

# Muat datanya
try:
    data=pd.read_csv('credit_scoring_eng.csv')
except:
    data = pd.read_csv('/datasets/credit_scoring_eng.csv')

# 3.1.2 Eksplorasi data

**Deskripsi data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - status perkawinan
- `family_status_id` - pengidentifikasi untuk status perkawinan nasabah
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

In [11]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
data.shape

(21525, 12)

In [12]:
# Mari tampilkan N baris pertama
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Jika diperhatikan pada kolom **days_employed** terdapat nilai minus (yang seharusnya tidak ada nilai minus dalam hari), perlu diselidiki lebih lagi apakah nilai tersebut mempengaruhi kolom yang lain.
Terdapat input nilai yang tidak standar pada kolom **education**, diperlukan standarisasi penulisan untuk hasil report yang rapi nantinya.

In [13]:
# Dapatkan informasi data
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Seperti yang kita lihat, data yang tersedia seharusnya memiliki **21525 record data**, namun terdapat nilai yang hilang di kolom days_employed dan total_income yang hanya memiliki **19351**.
Perlunya pemeriksaan lebih lanjut untuk nilai yang hilang tersebut.

In [14]:
# Mari kita lihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang
data.loc[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


- Nilai yang hilang pada kolom tersebut simetris
- Untuk nilai hilang di kolom **days_employed** diikuti juga dengan hilangnya nilai di kolom **total_income**.

In [15]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [16]:
# Persentase data yang hilang
data.isna().sum().value_counts(normalize=True) * 100

0       83.333333
2174    16.666667
dtype: float64

## 3.1.3 Kesimpulan sementara

Findings:
 - Data yang kita miliki terdiri dari 21525 baris data dan 12 kolom
 - Terdapat nilai yang tidak sesuai di kolom **days_employed** dan **education**
 - Terdapat nilai hilang dikolom **days_employed** dan **total_income**
 - Persentase data hilang hampir mencapai 17%
 
 
Insights:
 - Isi data yang kita punya mewakili informasi [ribadi nasabah

Recommendation:

 - Diperlukannya penanganan untuk perbaikan data pada kolom, dan nilai yang hilang agar hasil analisa sesuai yang diharapkan.

# 3.2 Menemukan pola nilai yang hilang

In [17]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
data_null=data.loc[(data['days_employed'].isnull())|(data['total_income'].isnull())].reset_index(drop=True)
data_null.shape

(2174, 12)

In [18]:
# Cek data_null
data_null.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
1,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
2,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
3,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
4,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [19]:
# Periksalah distribusinya
data_null['children'].value_counts(normalize=True)*100

 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 20     0.413983
 4      0.321987
-1      0.137994
 5      0.045998
Name: children, dtype: float64

In [20]:
data_null['dob_years'].value_counts(normalize=True)*100

34    3.173873
40    3.035879
42    2.989880
31    2.989880
35    2.943882
36    2.897884
47    2.713891
41    2.713891
30    2.667893
28    2.621895
58    2.575897
57    2.575897
54    2.529899
56    2.483901
38    2.483901
52    2.437902
37    2.437902
33    2.345906
39    2.345906
50    2.345906
43    2.299908
45    2.299908
49    2.299908
51    2.299908
29    2.299908
46    2.207912
55    2.207912
48    2.115915
44    2.023919
53    2.023919
60    1.793928
62    1.747930
61    1.747930
32    1.701932
64    1.701932
23    1.655934
27    1.655934
26    1.609936
59    1.563937
63    1.333947
25    1.057958
24    0.965961
65    0.919963
66    0.919963
21    0.827967
22    0.781969
67    0.735971
0     0.459982
68    0.413983
71    0.229991
69    0.229991
20    0.229991
70    0.137994
72    0.091996
19    0.045998
73    0.045998
Name: dob_years, dtype: float64

In [21]:
data_null['education'].value_counts(normalize=True)*100

secondary education    64.765409
bachelor's degree      22.815087
SECONDARY EDUCATION     3.081877
Secondary Education     2.989880
some college            2.529899
Bachelor's Degree       1.149954
BACHELOR'S DEGREE       1.057958
primary education       0.873965
Some College            0.321987
SOME COLLEGE            0.321987
Primary Education       0.045998
PRIMARY EDUCATION       0.045998
Name: education, dtype: float64

In [22]:
data_null['education_id'].value_counts(normalize=True)*100

1    70.837167
0    25.022999
2     3.173873
3     0.965961
Name: education_id, dtype: float64

In [23]:
data_null['family_status'].value_counts(normalize=True)*100

married              56.899724
civil partnership    20.331187
unmarried            13.247470
divorced              5.151794
widow / widower       4.369825
Name: family_status, dtype: float64

In [25]:
data_null['family_status_id'].value_counts(normalize=True)*100

0    56.899724
1    20.331187
4    13.247470
3     5.151794
2     4.369825
Name: family_status_id, dtype: float64

In [26]:
data_null['gender'].value_counts(normalize=True)*100

F    68.26127
M    31.73873
Name: gender, dtype: float64

In [27]:
data_null['income_type'].value_counts(normalize=True)*100

employee         50.827967
business         23.367065
retiree          18.997240
civil servant     6.761730
entrepreneur      0.045998
Name: income_type, dtype: float64

In [28]:
data_null['debt'].value_counts(normalize=True)*100

0    92.180313
1     7.819687
Name: debt, dtype: float64

In [29]:
data_null['purpose'].value_counts(normalize=True)*100

having a wedding                            4.231831
to have a wedding                           3.725851
wedding ceremony                            3.495860
construction of own property                3.449862
housing transactions                        3.403864
buy real estate                             3.311868
purchase of the house for my family         3.265869
transactions with my real estate            3.265869
transactions with commercial real estate    3.219871
housing renovation                          3.219871
buy commercial real estate                  3.081877
buying property for renting out             2.989880
property                                    2.851886
real estate transactions                    2.805888
buy residential real estate                 2.805888
housing                                     2.759890
building a property                         2.713891
cars                                        2.621895
going to university                         2.

Setelah kita cek distribusi data terhadap nilai yang hilang berikut yang bisa saya sampaikan atas temuan saya:

- untuk nilai yang hilang di dominasi oleh data yang:
  - tidak memiliki anak
  - umur 34 tahun
  - married
  - female
  - employee

In [30]:
# Memeriksa distribusi di seluruh dataset
data['children'].value_counts(normalize=True)*100

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

In [31]:
data['dob_years'].value_counts(normalize=True)*100

35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2.601626
36    2.578397
44    2.541231
29    2.531940
30    2.508711
48    2.499419
37    2.494774
50    2.387921
43    2.383275
32    2.369338
49    2.360046
28    2.336818
45    2.308943
27    2.290360
56    2.262485
52    2.248548
47    2.229965
54    2.225319
46    2.206736
58    2.141696
57    2.137050
53    2.132404
51    2.081301
59    2.062718
55    2.058072
26    1.895470
60    1.751452
25    1.658537
61    1.649245
62    1.635308
63    1.249710
64    1.231127
24    1.226481
23    1.180023
65    0.901278
22    0.850174
66    0.850174
67    0.775842
21    0.515679
0     0.469222
68    0.459930
69    0.394890
70    0.301974
71    0.269454
20    0.236934
72    0.153310
19    0.065041
73    0.037166
74    0.027875
75    0.004646
Name: dob_years, dtype: float64

In [32]:
data['education'].value_counts(normalize=True)*100

secondary education    63.879210
bachelor's degree      21.918699
SECONDARY EDUCATION     3.586527
Secondary Education     3.303136
some college            3.103368
BACHELOR'S DEGREE       1.272938
Bachelor's Degree       1.245064
primary education       1.161440
Some College            0.218351
SOME COLLEGE            0.134727
PRIMARY EDUCATION       0.078978
Primary Education       0.069686
graduate degree         0.018583
Graduate Degree         0.004646
GRADUATE DEGREE         0.004646
Name: education, dtype: float64

In [33]:
data['education_id'].value_counts(normalize=True)*100

1    70.768873
0    24.436702
2     3.456446
3     1.310105
4     0.027875
Name: education_id, dtype: float64

In [34]:
data['family_status'].value_counts(normalize=True)*100

married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64

In [35]:
data['family_status_id'].value_counts(normalize=True)*100

0    57.514518
1    19.405343
4    13.068525
3     5.551684
2     4.459930
Name: family_status_id, dtype: float64

In [36]:
data['gender'].value_counts(normalize=True)*100

F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64

In [37]:
data['income_type'].value_counts(normalize=True)*100

employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
unemployed                      0.009292
entrepreneur                    0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64

In [38]:
data['debt'].value_counts(normalize=True)*100

0    91.911731
1     8.088269
Name: debt, dtype: float64

In [39]:
data['purpose'].value_counts(normalize=True)*100

wedding ceremony                            3.702671
having a wedding                            3.609756
to have a wedding                           3.595819
real estate transactions                    3.140534
buy commercial real estate                  3.084785
buying property for renting out             3.033682
housing transactions                        3.033682
transactions with commercial real estate    3.024390
purchase of the house                       3.005807
housing                                     3.005807
purchase of the house for my family         2.977933
construction of own property                2.950058
property                                    2.945412
transactions with my real estate            2.926829
building a real estate                      2.908246
buy real estate                             2.898955
building a property                         2.880372
purchase of my own house                    2.880372
housing renovation                          2.

In [40]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
data_null.loc[data_null['income_type']=='employee']['children'].value_counts(normalize=True)*100

 0     58.914027
 1     25.248869
 2     12.398190
 3      2.624434
 4      0.361991
 20     0.271493
 5      0.090498
-1      0.090498
Name: children, dtype: float64

In [41]:
# Periksa pola lainnya - jelaskan pola tersebut
data_null.loc[data_null['income_type']=='employee']['family_status'].value_counts(normalize=True)*100

married              59.095023
civil partnership    20.361991
unmarried            13.031674
divorced              5.158371
widow / widower       2.352941
Name: family_status, dtype: float64

## 3.3 Transformasi data

In [42]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
data['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [43]:
# Perbaiki pencatatan jika diperlukan
data['education']=data['education'].str.lower()

In [44]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
data['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [45]:
# Mari kita lihat distribusi nilai pada kolom `children`
data['children'].value_counts(normalize=True)

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

- terdapat nilai aneh yang ada di kolom children (terdapat sebanyak 47  nilai yang berisi -1, karena tidak ada jumlah anak -1), 
- terdapat 1 nilai yaitu 20, sepertinya ini kesalahan input (karna kecil kemungkinan mempunyai anak sejumlah 20)
- untuk kedua nilai aneh tersebut akan dilakukan perubahan dari -1 menjadi 1 dan 20 menjadi 2

In [46]:
# memperbaiki data pada kolom children
data['children']=data['children'].replace([-1,20],[1,2])

In [47]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
data['children'].value_counts(normalize=True)

0    0.657329
1    0.226016
2    0.099001
3    0.015331
4    0.001905
5    0.000418
Name: children, dtype: float64

In [48]:
# Temukan data yang bermasalah di kolom `days_employed` jika memang terdapat masalah dan hitung persentasenya
data['days_employed'].value_counts(normalize=True)*100

-8437.673028      0.005168
-3507.818775      0.005168
 354500.415854    0.005168
-769.717438       0.005168
-3963.590317      0.005168
                    ...   
-1099.957609      0.005168
-209.984794       0.005168
 398099.392433    0.005168
-1271.038880      0.005168
-1984.507589      0.005168
Name: days_employed, Length: 19351, dtype: float64

- terdapat jumlah hari yang bernilai negatif

In [49]:
# Jumlah nilai yang negatif
(data['days_employed']<0).sum()

15906

In [50]:
# Jumlah nilai yang hilang di kolom days_employed
data['days_employed'].isna().sum()

2174

In [51]:
# Atasi nilai yang bermasalah, jika ada
data['days_employed']=data['days_employed'].abs()

- Perbaikan data yang bernilai negatif menjadi absolut

In [52]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
data['days_employed'].value_counts(normalize=True)*100

8437.673028      0.005168
3507.818775      0.005168
354500.415854    0.005168
769.717438       0.005168
3963.590317      0.005168
                   ...   
1099.957609      0.005168
209.984794       0.005168
398099.392433    0.005168
1271.038880      0.005168
1984.507589      0.005168
Name: days_employed, Length: 19351, dtype: float64

In [53]:
data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [54]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
data['dob_years'].value_counts(normalize=True).sort_index()

0     0.004692
19    0.000650
20    0.002369
21    0.005157
22    0.008502
23    0.011800
24    0.012265
25    0.016585
26    0.018955
27    0.022904
28    0.023368
29    0.025319
30    0.025087
31    0.026016
32    0.023693
33    0.026992
34    0.028014
35    0.028664
36    0.025784
37    0.024948
38    0.027782
39    0.026620
40    0.028293
41    0.028200
42    0.027735
43    0.023833
44    0.025412
45    0.023089
46    0.022067
47    0.022300
48    0.024994
49    0.023600
50    0.023879
51    0.020813
52    0.022485
53    0.021324
54    0.022253
55    0.020581
56    0.022625
57    0.021370
58    0.021417
59    0.020627
60    0.017515
61    0.016492
62    0.016353
63    0.012497
64    0.012311
65    0.009013
66    0.008502
67    0.007758
68    0.004599
69    0.003949
70    0.003020
71    0.002695
72    0.001533
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64

- Terdapat umur yang nilainya 0 dalam data
- nantinya nilai ini akan digantikan dengan nilai median dari keseluruhan data

In [55]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
# Menghitung median dari keseluruhan data kolom 'dob_years'
median = data['dob_years'].median()
median

42.0

In [56]:
# Mengganti nilai 0 di kolom 'dob_years' dengan nilai median
data['dob_years'].replace(0, median, inplace=True)

In [57]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [58]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
data['dob_years'].isnull().sum()

data['dob_years'].value_counts(normalize=True).sort_index()

19    0.000650
20    0.002369
21    0.005157
22    0.008502
23    0.011800
24    0.012265
25    0.016585
26    0.018955
27    0.022904
28    0.023368
29    0.025319
30    0.025087
31    0.026016
32    0.023693
33    0.026992
34    0.028014
35    0.028664
36    0.025784
37    0.024948
38    0.027782
39    0.026620
40    0.028293
41    0.028200
42    0.032427
43    0.023833
44    0.025412
45    0.023089
46    0.022067
47    0.022300
48    0.024994
49    0.023600
50    0.023879
51    0.020813
52    0.022485
53    0.021324
54    0.022253
55    0.020581
56    0.022625
57    0.021370
58    0.021417
59    0.020627
60    0.017515
61    0.016492
62    0.016353
63    0.012497
64    0.012311
65    0.009013
66    0.008502
67    0.007758
68    0.004599
69    0.003949
70    0.003020
71    0.002695
72    0.001533
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64

In [59]:
# Mari kita lihat nilai untuk kolom ini
data['family_status'].value_counts(normalize=True)*100

married              57.514518
civil partnership    19.405343
unmarried            13.068525
divorced              5.551684
widow / widower       4.459930
Name: family_status, dtype: float64

In [60]:
# Mari kita liat nilai dalam kolom ini
data['gender'].value_counts(normalize=True)*100

F      66.137050
M      33.858304
XNA     0.004646
Name: gender, dtype: float64

- Pada kolom **'gender'** terdapat nilai **'XNA'**.
- Untuk nilai tersebut akan di hapus dikarenakan hanya terdapat 1 baris data saja, dan tidak mempengaruhi analisa nantinya

In [61]:
# Atasi nilai-nilai yang bermasalah, jika ada
data=data.drop(data.loc[data['gender'] == 'XNA'].index)

In [62]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
sorted(data['gender'].unique())

['F', 'M']

In [63]:
# Mari kita lihat nilai dalam kolom ini
data['income_type'].value_counts(normalize=True)*100

employee                       51.658614
business                       23.620145
retiree                        17.914886
civil servant                   6.778480
unemployed                      0.009292
entrepreneur                    0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64

# 3.4 Memperbaiki duplikasi data

In [64]:
# Periksa duplikat
data.duplicated().sum()

72

In [65]:
# Atasi duplikat, jika ada
data=data.drop_duplicates()

In [66]:
# Lakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
data.duplicated().sum()

0

In [67]:
# Periksa ukuran dataset yang sekarang kamu miliki setelah manipulasi pertama yang kamu lakukan
data.shape

(21452, 12)

Findings:

  -  Dataset saat ini menggunakan variabel data
  -  terdapat nilai yang tidak sesuai di kolom gender dan dob_years
  -  terdapat duplikasi data

Insight:

  -  Sudah dilakukan Perubahan nilai di kolom **'children'** yang -1 dan 20 menjadi 1 dan 2 (perubahan dilakukan karena tidak ada anak sejumlah -1 dan juga saat ini kecil kemungkinan mempunyai anak sejumlah 20, bisa jadi dikarenakan typo ketika input data.
  - dari perubahan nilai negatif di kolom **days_employed** menjadi positif
  - penggantian nilai 0 di dalam kolom **dob_years** dengan nilai median dari keseluruhan data pada kolom tesebut.
  - menghapus nilai 'XNA' di dalam kolom **gender**
  - hingga penghapusan data duplicate dalam dataset

Recommendation:

  - Pemeriksaan lanjutan untuk dataset yang kita punya, seperti nilai yang hilang

# 3.5 Bekerja dengan nilai data

Selanjutnya kita akan memerlukan dictionary untuk mempermudah dalam hal menemukan kesalahan/ nilai yang hilang pada data yang kita punya, pada data kita memerlukan kolom yang berisikan nilai unik (tidak boleh sama) dalam hal ini **'education_id'** dan **'family_status_id'** diperlukan untuk membantu kita menemukan nilai yang hilang nantinya.

In [68]:
data[['days_employed', 'total_income']].describe()

,days_employed,total_income
count,19350.000000,19350.000000
mean,66918.065141,26787.266688
std,139033.698578,16475.822926
min,24.141633,3306.762000
25%,926.990457,16486.515250
50%,2194.218768,23201.873500
75%,5538.423086,32547.910750
max,401755.400475,362496.645000


In [69]:
# Temukan dictionary
dict_edu=data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

In [70]:
dict_edu.to_dict(orient='records' )

[{'education_id': 0, 'education': "bachelor's degree"},
 {'education_id': 1, 'education': 'secondary education'},
 {'education_id': 2, 'education': 'some college'},
 {'education_id': 3, 'education': 'primary education'},
 {'education_id': 4, 'education': 'graduate degree'}]

In [71]:
dict_family=data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

In [72]:
dict_family.to_dict(orient='records' )

[{'family_status_id': 0, 'family_status': 'married'},
 {'family_status_id': 1, 'family_status': 'civil partnership'},
 {'family_status_id': 2, 'family_status': 'widow / widower'},
 {'family_status_id': 3, 'family_status': 'divorced'},
 {'family_status_id': 4, 'family_status': 'unmarried'}]

### 3.5.1 Memperbaiki nilai yang hilang di `total_income`

Sebelumnya sudah kita lakukan pengecekan pada data diatas, bahwa pada kolom **'days_employed'** dan **'total_income** terdapat data yang hilang yang tidak dengan serta merta dapat kita hapus karena nilai hilang itu cukup besar untuk menjadi acuan analisa nantinya.

Saat ini kita akan menangani terlebih dahulu nilai hilang yang berada di kolom **'total_income'**. Kita akan membuat kategori untuk nasabah untuk antinya dapat membantu kita untuk menghitung nilai pendapatan


In [73]:
# Cek rentang usia
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [74]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def age_grouping(age):
    if age <=20:
        value='0-20 years old'
    elif age >=21 and age <=45:
        value='21-45 years old'
    elif age >=46 and age <=65:
        value='46-65 years old'
    else:
        value='> 65 years old'
    return(value)   

In [75]:
# Lakukan pengujian untuk melihat apakah fungsimu bekerja atau tidak
age_grouping(30)

'21-45 years old'

In [76]:
# Buatlah kolom baru berdasarkan fungsi
data['age_group'] = data['dob_years'].apply(age_grouping)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old


In [77]:
# Periksa bagaimana nilai di dalam kolom baru
data['age_group'].value_counts()

21-45 years old    12237
46-65 years old     8448
> 65 years old       702
0-20 years old        65
Name: age_group, dtype: int64

In [78]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
not_null_income=data.loc[~data['total_income'].isnull()]
not_null_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,21-45 years old
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 65 years old
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,21-45 years old
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,21-45 years old


In [79]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah kamu identifikasi
not_null_income.groupby('age_group')['total_income'].mean()

age_group
0-20 years old     19586.303559
21-45 years old    27705.426256
46-65 years old    26012.644119
> 65 years old     20804.461461
Name: total_income, dtype: float64

In [80]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah kamu identifikasi
age_income=pd.pivot_table(not_null_income, index='age_group', values='total_income', aggfunc='median').reset_index()
age_income

,age_group,total_income
0,0-20 years old,17257.277
1,21-45 years old,24102.286
2,46-65 years old,22471.631
3,> 65 years old,17978.231


In [81]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang
def replace_income(dataset, grouping):
    for group in grouping['age_group']:
        index = 0
        dataset.loc[(dataset['age_group']==group) & (dataset['total_income'].isnull()), 'total_income'] = grouping.iloc[index, 1]
        index = index + 1
    return dataset

In [82]:
# Memeriksa bagaimana nilai di dalam kolom baru
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old


In [83]:
# Terapkan fungsi tersebut ke setiap baris
replace_income(dataset=data, grouping=age_income)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,21-45 years old
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 65 years old
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,21-45 years old
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,21-45 years old


In [84]:
# Periksa apakah kita mendapatkan kesalahan
data.isnull().sum()

children               0
days_employed       2102
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
dtype: int64

In [85]:
# Periksa jumlah entri di kolom
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21452 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
 12  age_group         21452 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


###  3.5.2 Memperbaiki nilai di `days_employed`

Dalam data yag kita punya, untuk 'days_employed' terdapat hari yang tidak wajar, perlu dilakukan konversi days_employed menjadi pengalaman kerja 'years_employed' untuk melihat pengalaman kerja dari data yang kita punya

In [86]:
# Mengisi data di 'dob_years' yang bernilai 0 dengan nilai mean
avg_age=int(data['dob_years'].mean())
data.loc[data['dob_years']==0, 'dob_years']=avg_age 

In [87]:
# Mengganti value pengalaman kerja yang tidak masuk akal
data['years_employed']=round(data['days_employed']/365)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0


terdapat data yang memiliki pengalaman kerja ratusan tahun, nilai ini nantinya akan digantikan dengan nilai yang wajar.

In [88]:
# Mendapatkan durasi bekerja yang sesungguhnya (umur bekerja dimulai 18 tahun)
data['years_working']=data['dob_years']- 18
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35


In [89]:
data.loc[data['years_working']<0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working


In [90]:
# Perbandingan 'years_employed' dengan 'years_working yang tidak wajar
data.loc[data['years_employed']>data['years_working']]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,21-45 years old,19.0,17
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car,46-65 years old,1097.0,35
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate,46-65 years old,928.0,39
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate,> 65 years old,996.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding,46-65 years old,929.0,35
21508,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property,46-65 years old,1059.0,44
21509,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions,46-65 years old,992.0,41
21518,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car,46-65 years old,1025.0,41


In [91]:
# Menganti 'days_employed' berdasarkan pengalaman yang sebenarnya
data.loc[data['years_employed']>data['years_working'],'days_employed']= data['years_working']*365
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14
4,0,12775.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35


In [92]:
data.isnull().sum()

children               0
days_employed       2102
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
years_employed      2102
years_working          0
dtype: int64

In [93]:
not_null_days=data.loc[~data['days_employed'].isnull()]

In [94]:
# Distribusi median dari `days_employed` berdasarkan parameter yang kamu identifikasi
age_days=pd.pivot_table(not_null_days, index='age_group', values='days_employed', aggfunc='median').reset_index()
age_days

,age_group,days_employed
0,0-20 years old,512.816111
1,21-45 years old,1462.641454
2,46-65 years old,4855.666725
3,> 65 years old,17885.000000


In [95]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang kamu identifikasi
not_null_days.groupby('age_group')['days_employed'].mean()

age_group
0-20 years old       510.975609
21-45 years old     2022.814926
46-65 years old     7400.293584
> 65 years old     16546.819846
Name: days_employed, dtype: float64

- Untuk nilai yang hilang pada kolom 'days_employed' nantinya akan diisi dengan nilai median, dikarenakan outlier dalam kolom tersebut perbedaannya cukup besar

In [96]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusanmu) berdasarkan parameter yang kamu identifikasi
def replace_days(dataset, grouping):
    for group in grouping['age_group']:
        index = 0
        dataset.loc[(dataset['age_group']==group) & (dataset['days_employed'].isnull()), 'days_employed'] = grouping.iloc[index, 1]
        index = index + 1
    return dataset

In [97]:
# Periksa apakah fungsimu bekerja
replace_days(dataset=data, grouping=age_days)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14
4,0,12775.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,21-45 years old,12.0,25
21521,0,17885.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 65 years old,942.0,49
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,21-45 years old,6.0,20
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,21-45 years old,9.0,20


In [98]:
# Periksa apakah fungsimu bekerja
data['days_employed'].isnull().sum()

0

In [99]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang
data.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
years_employed      2102
years_working          0
dtype: int64

# 3.6 Pengkategorian data


In [100]:
# Tampilkan nilai data yang kamu pilih untuk pengkategorian
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14
4,0,12775.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35


In [101]:
# Periksa nilai unik
data['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            767
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             620
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

- Diatas terdapat kita identifikasi nilai di kolom 'purpose' dan perlu disederhanakan untuk mempermudah analisa kita nantinya

In [102]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def categorize_purpose(row):
    if 'car' in row:
        return 'car'
    elif 'hous' in row or 'prop' in row or 'real est' in row:
        return 'real estate'
    elif 'wedd' in row:
        return 'wedding'
    elif 'educ' in row or 'uni' in row:
        return 'education'
    else:
        return 'Unidentified'

In [103]:
# Buat kolom yang memuat kategori dan hitung nilainya
data['purpose_category']=data['purpose'].apply(categorize_purpose)

In [104]:
# Lihat semua data numerik di kolom yang kamu pilih untuk pengkategorian
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24,real estate
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15,real estate
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14,education
4,0,12775.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35,wedding


In [105]:
# Dapatkan kesimpulan statistik untuk kolomnya
data['purpose_category'].value_counts(normalize=True)*100

real estate    50.391572
car            20.072720
education      18.706880
wedding        10.828827
Name: purpose_category, dtype: float64

In [106]:
# Buat fungsi yang melakukan pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang
def income_level(income):
    if (income > 5000) and (income <= 10000):
        return 'small'
    if (income > 10000) and (income <= 15000):
        return 'avarage'
    if (income > 20000) and (income <= 25000):
        return 'above avarege'
    if (income > 25000) and (income <= 50000):
        return 'high'
    if income > 50000:
        return 'very high'

In [107]:
# Buat kolom yang memuat kategori
data['income_category']=data['total_income'].apply(income_level)

In [108]:
# Hitung setiap nilai kategori untuk melihat pendistribusiannya
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,years_employed,years_working,purpose_category,income_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,21-45 years old,23.0,24,real estate,high
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21-45 years old,11.0,18,car,None
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21-45 years old,15.0,15,real estate,above avarege
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21-45 years old,11.0,14,education,high
4,0,12775.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65 years old,932.0,35,wedding,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,21-45 years old,12.0,25,real estate,high
21521,0,17885.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,> 65 years old,942.0,49,car,above avarege
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,21-45 years old,6.0,20,real estate,avarage
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,21-45 years old,9.0,20,car,high


## 3.7 Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [109]:
# Periksa data anak dan data gagal bayar pinjaman
pd.pivot_table(data,index='debt', columns='children', values='education',aggfunc='count')

children,0,1,2,3,4,5
debt,,,,,,
0,13026.0,4410.0,1926.0,303.0,37.0,9.0
1,1063.0,445.0,202.0,27.0,4.0,NaN


In [110]:
# Mengganti nilai NaN diisi dengan nilai yang kita mau, dalam hal ini diisi 0
pd.pivot_table(data,index='debt', columns='children', values='education',aggfunc='count').fillna(0)

children,0,1,2,3,4,5
debt,,,,,,
0,13026.0,4410.0,1926.0,303.0,37.0,9.0
1,1063.0,445.0,202.0,27.0,4.0,0.0


Dikarenakan kita mau menghitung perentase dari gagal bayar tampilan Pivot tabel diubah menjadi seperti ini

In [111]:
# Untuk perhitungan persentase tampilan Pivot table dibuat seperti ini agar memudahkan untuk membaca informasinya
children_debt=pd.pivot_table(data,index='children', columns='debt', values='education',aggfunc='count').fillna(0).reset_index()
children_debt

debt,children,0,1
0,0,13026.0,1063.0
1,1,4410.0,445.0
2,2,1926.0,202.0
3,3,303.0,27.0
4,4,37.0,4.0
5,5,9.0,0.0


In [112]:
# Hitung persentase gagal bayar berdasarkan jumlah anak
children_debt['total']=children_debt[0]+children_debt[1]
children_debt['percent 0']=round(children_debt[0]/children_debt['total']*100,1).astype(str)+'%'
children_debt['percent 1']=round(children_debt[1]/children_debt['total']*100,1).astype(str)+'%'
children_debt.sort_values('percent 1',ascending=False)

debt,children,0,1,total,percent 0,percent 1
4,4,37.0,4.0,41.0,90.2%,9.8%
2,2,1926.0,202.0,2128.0,90.5%,9.5%
1,1,4410.0,445.0,4855.0,90.8%,9.2%
3,3,303.0,27.0,330.0,91.8%,8.2%
0,0,13026.0,1063.0,14089.0,92.5%,7.5%
5,5,9.0,0.0,9.0,100.0%,0.0%


**Kesimpulan**

Artinya, untuk kelompok dengan:
- 0 anak, persentase gagal bayar adalah 7,5%
- 1 anak, persentase gagal bayar adalah 9,2%
- 2 anak, persentase gagal bayar adalah 9,5%
- 3 anak, persentase gagal bayar adalah 8,2%
- 4 anak, persentase gagal bayar adalah 9,8%.
- 5 anak, persentase gagal bayar adalah 0%.

- Dari data diatas dapat saya simpulkan, jumlah anak tidak mempengaruhi gagal bayar seorang nasabah, dikarenakan untuk nasabah yang tidak memiliki anak persentase gagal bayarnyapun tidak berbeda jauh dengan nasabah yang memiliki 1 nak bahkan lebih.

- sedangkan Untuk nasabah yang memiliki 5 anak tidak ada yang mengalami keterlambatan pembayaran dan nasabah yang memiliki 4 anak menjadi persentase tertinggi dalam gagal bayar.


**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [113]:
# Periksa data status keluarga dan data gagal bayar pinjaman
pd.pivot_table(data,index='debt', columns='family_status', values='education',aggfunc='count')

family_status,civil partnership,divorced,married,unmarried,widow / widower
debt,,,,,
0,3761,1110,11408,2536,896
1,388,85,931,274,63


In [114]:
# Untuk perhitungan persentase tampilan Pivot table dibuat seperti ini agar memudahkan untuk membaca informasinya
family_status_debt=pd.pivot_table(data,index='family_status', columns='debt', values='education',aggfunc='count').reset_index()
family_status_debt

debt,family_status,0,1
0,civil partnership,3761,388
1,divorced,1110,85
2,married,11408,931
3,unmarried,2536,274
4,widow / widower,896,63


In [115]:
# Hitung persentase gagal bayar berdasarkan status keluarga
family_status_debt['total']=family_status_debt[0]+family_status_debt[1]
family_status_debt['percent 0']=round(family_status_debt[0]/family_status_debt['total']*100,1).astype(str)+'%'
family_status_debt['percent 1']=round(family_status_debt[1]/family_status_debt['total']*100,1).astype(str)+'%'
family_status_debt.sort_values('percent 1',ascending=False)

debt,family_status,0,1,total,percent 0,percent 1
3,unmarried,2536,274,2810,90.2%,9.8%
0,civil partnership,3761,388,4149,90.6%,9.4%
2,married,11408,931,12339,92.5%,7.5%
1,divorced,1110,85,1195,92.9%,7.1%
4,widow / widower,896,63,959,93.4%,6.6%


**Kesimpulan**

Artinya, untuk kelompok dengan Status keluarga:
- civil partnership, persentase gagal bayar adalah 9,4%
- divorced, persentase gagal bayar adalah 7,1%
- married, persentase gagal bayar adalah 7,5%
- unmarried, persentase gagal bayar adalah 9,8%
- widow / widower, persentase gagal bayar adalah 6,6%.


Untuk fmily status unmarried mendominasi gagalnya pembayaran dengan persentase 9,8%, itu berarti untuk nasabah yang belum menikah memiliki persentase gagal bayar lebih besar dari status keluarga yang lainnya

**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [116]:
# Periksa data tingkat pendapatan dan data gagal bayar pinjaman
pd.pivot_table(data,index='debt', columns='income_category', values='education',aggfunc='count')

income_category,above avarege,avarage,high,small,very high
debt,,,,,
0,3091,2577,6699,844,1228
1,288,240,583,56,92


In [117]:
# Untuk perhitungan persentase tampilan Pivot table dibuat seperti ini agar memudahkan untuk membaca informasinya
category_income_debt=pd.pivot_table(data,index='income_category', columns='debt', values='education',aggfunc='count').reset_index()
category_income_debt

debt,income_category,0,1
0,above avarege,3091,288
1,avarage,2577,240
2,high,6699,583
3,small,844,56
4,very high,1228,92


In [118]:
data[['total_income','income_category']]

,total_income,income_category
0,40620.102,high
1,17932.802,None
2,23341.752,above avarege
3,42820.568,high
4,25378.572,high
...,...,...
21520,35966.698,high
21521,24959.969,above avarege
21522,14347.610,avarage
21523,39054.888,high


In [119]:
# Hitung persentase gagal bayar berdasarkan tingkat pendapatan
category_income_debt['total']=category_income_debt[0]+category_income_debt[1]
category_income_debt['percent 0']=round(category_income_debt[0]/category_income_debt['total']*100,1).astype(str)+'%'
category_income_debt['percent 1']=round(category_income_debt[1]/category_income_debt['total']*100,1).astype(str)+'%'
category_income_debt.sort_values('percent 1',ascending=False)

debt,income_category,0,1,total,percent 0,percent 1
0,above avarege,3091,288,3379,91.5%,8.5%
1,avarage,2577,240,2817,91.5%,8.5%
2,high,6699,583,7282,92.0%,8.0%
4,very high,1228,92,1320,93.0%,7.0%
3,small,844,56,900,93.8%,6.2%


**Kesimpulan**

- Dari keterangan diatas, semakin tingginya pendapatan nasabah membuat dia kemungkinan besar terjadi gagal bayar

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [120]:
# # Periksa data tingkat pendapatan dan data gagal bayar pinjaman
purpose_debt=pd.pivot_table(data,index='purpose_category', columns='debt', values='education',aggfunc='count').reset_index()
purpose_debt

debt,purpose_category,0,1
0,car,3903,403
1,education,3643,370
2,real estate,10028,782
3,wedding,2137,186


In [121]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
purpose_debt['total']=purpose_debt[0]+purpose_debt[1]
purpose_debt['percent 0']=round(purpose_debt[0]/purpose_debt['total']*100,1).astype(str)+'%'
purpose_debt['percent 1']=round(purpose_debt[1]/purpose_debt['total']*100,1).astype(str)+'%'
purpose_debt.sort_values('percent 1',ascending=False)

debt,purpose_category,0,1,total,percent 0,percent 1
0,car,3903,403,4306,90.6%,9.4%
1,education,3643,370,4013,90.8%,9.2%
3,wedding,2137,186,2323,92.0%,8.0%
2,real estate,10028,782,10810,92.8%,7.2%


**Kesimpulan**

Dari keterangan diatas, penggunaan kredit yang mengakibatkan nasabah gagal bayar lebih banyak digunakan untuk **car** dan **education**


# 3.8 Kesimpulan umum 

Dari data yang saya dapatkan. terdapat nilai yang hilang dalam data tersebut, juga terdapat nilai-nilai yang tidak wajar. seperti jumlah anak yang bernilai negatif dan jumlah anak yang tidak wajar sampai 20 anak. pada total income juga saya mendapati jumlah pedapatan yang bernilai negatif.

pada kolom days_employed terdapat jumlah nilai yang tidak wajar, dimana jika di konversi ke tahun jumlah kerja nasabah bisa sampai ratusan tahun.

itu semua ada kemungkinan kesalahan sistem dan human error yang terjadi dalampenginputan data, namun setelah ssaya perbaiki kiranya laporan ini dapat menjadi acuan untuk pemberian kredit kepada calon nasabah nantinya

- Apakah terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman?

  - Dari data diatas dapat saya simpulkan, jumlah anak tidak mempengaruhi gagal bayar seorang nasabah, karena dengan banyaknya anak (dalam hal ini 5 anak) tidak mengalami permasalahan dalam gagal bayar, justru yang tidak memiliki anak berpotensi terjadinya gagal bayar.


- Apakah terdapat hubungan antara status perkawinan dan probabilitas seseorang melakukan gagal bayar pinjaman?

  - Ketika seorang nasabah belum mempunyai **status keluarga (unmarried)** berpotensi menjadi calon nasabah yang nantinya akan terjadi gagal bayar.



- Apakah terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman?

  - **Semakin tingginya pendapatan** nasabah membuat dia kemungkinan besar terjadi gagal bayar.


- Bagaimana perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman?
 
  - Dalam tujuan peminjaman terdapat 2 kategori yang mendominasi terjadinya gagal bayar. Dalam hal ini nasabah sebagian besar lebih banyak menggunakan dana pinjaman untuk membeli **mobil**, selanjutnya tidak digunakan untuk **biaya pendidikan**.
